In [1]:
# import library
import string
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from util import JSONParser

In [2]:
# load data
path = "data/intents.json"

# buat objek JSONParser dan parse data intents.json
jp = JSONParser()
jp.parse(path)

# simpan dataframe dalam variabel df
df = jp.get_dataframe()

[INFO] Data JSON converted to DataFrame with shape : (1698, 2)


In [3]:
# lihat 5 data pertama
df.head()

,text_input,intents
0,"saya memiliki masalah sengketa harta waris, ba...",gugatan_waris_sengketa_harta
1,bagaimana proses masalah sengketa waris?,gugatan_waris_sengketa_harta
2,proses sengketa harta waris,gugatan_waris_sengketa_harta
3,proses sengketa waris,gugatan_waris_sengketa_harta
4,sengketa harta waris,gugatan_waris_sengketa_harta


In [4]:
# hitung jumlah data per tag / inten
df.intents.value_counts()

info_umum                             164
tinjauan_kembali                       68
permohonan_form_nikah_dibawah_umur     20
permohonan_isbat_volunteer             19
permohonan_form_dispen_nikah           18
                                     ... 
ecourt_definisi                         3
ecourt_e-litigation                     3
ecourt_e-summons                        3
info_umum_permohonan                    3
gugatan_sederhana_definisi              2
Name: intents, Length: 241, dtype: int64

In [5]:
def preprocess(chat):
    """
    Fungsi yang digunakan untuk melakukan praproses
    """
    # konversi ke lowercase
    chat = chat.lower()
    # menghapus tanda baca
    tandabaca = tuple(string.punctuation)
    chat = ''.join(ch for ch in chat if ch not in tandabaca)
    return chat

In [6]:
# implementasikan fungsi preprocess ke string
df['text_input_prep'] = df.text_input.apply(preprocess)

Apabila kita lihat hasilnya maka kita dapati hal berikut : 

In [7]:
df[['text_input', 'text_input_prep']].head(10)

,text_input,text_input_prep
0,"saya memiliki masalah sengketa harta waris, ba...",saya memiliki masalah sengketa harta waris bag...
1,bagaimana proses masalah sengketa waris?,bagaimana proses masalah sengketa waris
2,proses sengketa harta waris,proses sengketa harta waris
3,proses sengketa waris,proses sengketa waris
4,sengketa harta waris,sengketa harta waris
5,sengketa waris,sengketa waris
6,masalah harta waris,masalah harta waris
7,bagaimana jika terjadi sengketa tanah waris ya...,bagaimana jika terjadi sengketa tanah waris ya...
8,sengketa tanah waris beda kota,sengketa tanah waris beda kota
9,sengketa tanah waris tempatnya jauh,sengketa tanah waris tempatnya jauh


In [8]:
# inisiasi objek CountVectorizer
vect = CountVectorizer()

In [9]:
# mengumpulkan vocab dari data teks yang sudah dilakukan praproses
vect.fit(df['text_input_prep'])

CountVectorizer()

In [10]:
# lihat list vocab
vect.get_feature_names()[:10] #batasi hanya 10 vocab teratas

c:\Users\fawza\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['18',
 '200',
 '21',
 'acara',
 'account',
 'action',
 'ada',
 'adakah',
 'adanya',
 'adhal']

In [11]:
# ubah data teks menjadi matriks
text_vect = vect.transform(df.text_input_prep)

text_vect

<1698x893 sparse matrix of type '<class 'numpy.int64'>'
	with 8421 stored elements in Compressed Sparse Row format>

In [12]:
pd.DataFrame(text_vect.toarray(), columns=vect.get_feature_names())

,18,200,21,acara,account,action,ada,adakah,adanya,adhal,...,wali,waris,warisan,warisnya,wasiat,wewenang,yang,yatim,yurisdiksi,zakat
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1694,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# deklarasi objek MultinomialNB
nb = MultinomialNB()

# training data, dengan X : text_vect dan y : intents
nb.fit(text_vect, df.intents)

MultinomialNB()

In [14]:
"""# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict(chat)

# tampilkan hasil prediksi
print(f"Hasil prediksi : {res[0]}")"""

'# input string dari user\nchat = input("Masukkan String : ")\n\n# lakukan preproses\nchat = preprocess(chat)\n\n# ubah teks menjadi vektor\nchat = vect.transform([chat])\n\n# prediksi vektor teks kedalam model machine learning\nres = nb.predict(chat)\n\n# tampilkan hasil prediksi\nprint(f"Hasil prediksi : {res[0]}")'

In [15]:
"""# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# ubah teks menjadi vektor
chat = vect.transform([chat])

# prediksi vektor teks kedalam model machine learning
res = nb.predict_proba(chat)

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")"""

'# input string dari user\nchat = input("Masukkan String : ")\n\n# lakukan preproses\nchat = preprocess(chat)\n\n# ubah teks menjadi vektor\nchat = vect.transform([chat])\n\n# prediksi vektor teks kedalam model machine learning\nres = nb.predict_proba(chat)\n\n# ambil nilai probabilitas tertinggi\nmax_prob = max(res[0])\nmax_idx = np.argmax(res[0])\nprint(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")'

In [16]:
# Deklarasi pipeline yang mengandung vektorisasi (CountVectorizer) & pemodelan (MultinomialNB) 
pipe = make_pipeline(CountVectorizer(),
                     MultinomialNB())

# Training
pipe.fit(df.text_input, df.intents)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [17]:
"""# input string dari user
chat = input("Masukkan String : ")

# lakukan preproses
chat = preprocess(chat)

# prediksi teks kedalam pipeline
res = pipe.predict_proba([chat])

# ambil nilai probabilitas tertinggi
max_prob = max(res[0])
max_idx = np.argmax(res[0])
print(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")"""

'# input string dari user\nchat = input("Masukkan String : ")\n\n# lakukan preproses\nchat = preprocess(chat)\n\n# prediksi teks kedalam pipeline\nres = pipe.predict_proba([chat])\n\n# ambil nilai probabilitas tertinggi\nmax_prob = max(res[0])\nmax_idx = np.argmax(res[0])\nprint(f"Max Prob : {max_prob}\nMax Index: {max_idx}\nLabel: {nb.classes_[max_idx]}")'

In [18]:
print("Anda Terhubung dengan chatbot Kami")
while True:
    # input user
    chat = input("Anda : ")
    # praproses
    chat = preprocess(chat)
    # prediksi intent
    res = pipe.predict_proba([chat])
    # ambil nilai probabilitas & lokasinya
    max_prob = max(res[0])
    max_idx = np.argmax(res[0])
    # kondisi jika probabilitas kurang dari threshold
    if max_prob < 0.07:
        print("Bot : Maaf, saya belum mengerti")
    else:
        print(f"Bot : {jp.get_response(nb.classes_[max_idx])}")
    if nb.classes_[max_idx] == 'selesai':
        break
    

Anda Terhubung dengan chatbot Kami
Bot : Formulir/Blanko untuk Pengangkatan Anak dapat diunduh pada tautan berikut: https://www.pa-bandung.go.id/images/File_Upload/POSBAKUM/BLANKO_PENGANGKATAN_ANAK.pdf
Bot : Formulir/Blanko untuk Pengangkatan Anak dapat diunduh pada tautan berikut: https://www.pa-bandung.go.id/images/File_Upload/POSBAKUM/BLANKO_PENGANGKATAN_ANAK.pdf
Bot : Rasulullah SAW Bersabda : "Sesuatu yang halal namun paling dibenci oleh Allah adalah perceraian." Apakah anda sudah yakin untuk melakukan perceraian? Bila sudah yakin, untuk mengajukan permohonan dispensasi nikah silahkan akses tautan berikut: http://gugatanmandiri.badilag.net/gugatan/mulai
Bot : k
Bot : Rasulullah SAW Bersabda : "Sesuatu yang halal namun paling dibenci oleh Allah adalah perceraian." Apakah anda sudah yakin untuk melakukan perceraian? Bila sudah yakin, untuk mengajukan permohonan dispensasi nikah silahkan akses tautan berikut: http://gugatanmandiri.badilag.net/gugatan/mulai
Bot : Persyaratan yang disi

KeyboardInterrupt: Interrupted by user

Setelah kita berhasil simulasikan bot nya dalam notebook ini, kita simpan modelnya agar dapat dideploy dengan mudah

In [ ]:
with open("chatbot_pipeline.pkl", "wb") as model_file:
    pickle.dump(pipe, model_file)